Import 

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn import metrics
import pandas_datareader.data as web
from time import time
from sklearn.model_selection import KFold

Data preparation

In [19]:
# Get Data
# Read stock data use pandas_datareader.data from web
# Get Stock Data

stk_tickers = ['MSFT', 'IBM', 'GOOGL']
ccy_tickers = ['DEXJPUS', 'DEXUSUK']
idx_tickers = ['SP500', 'DJIA', 'VIXCLS']
stk_data = web.DataReader(stk_tickers, 'yahoo')
ccy_data = web.DataReader(ccy_tickers, 'fred')
idx_data = web.DataReader(idx_tickers, 'fred')
# Select columns
base = stk_data.loc[:, ('Adj Close', 'MSFT')]
X1 = stk_data.loc[:, ('Adj Close', ('GOOGL', 'IBM'))]
X2 = ccy_data
X3 = idx_data

# Standardize Data
# Standardized data (X1, X2, X3) with kept index (date)
standard_scaler = preprocessing.StandardScaler()
std = standard_scaler.fit_transform(X1.values)
std2 = standard_scaler.fit_transform(X2.values)
std3 = standard_scaler.fit_transform(X3.values)
X1 = pd.DataFrame(data=std,index=X1.index,columns=X1.columns)
X2 = pd.DataFrame(data=std2,index=X2.index,columns=X2.columns)
X3 = pd.DataFrame(data=std3,index=X3.index,columns=X3.columns)
X1

Attributes Adj Close          
Symbols        GOOGL       IBM
Date                          
2017-09-25 -1.159659 -0.510365
2017-09-26 -1.154671 -0.462377
2017-09-27 -1.119086 -0.524971
2017-09-28 -1.111310 -0.524971
2017-09-29 -1.097200 -0.565305
...              ...       ...
2022-09-16  0.616757  1.088542
2022-09-19  0.625309  1.130989
2022-09-20  0.564180  0.999037
2022-09-21  0.505268  0.872622
2022-09-22  0.532031  0.957975

[1258 rows x 2 columns]

In [20]:
# Calculate ความแตกต่างของค่า ราคา 'Adj Close', 'MSFT’)ย้อนหลัง backHisotry วัน
backHistory = [30, 45, 60, 90, 180, 240] # -> ทดลองหยิบ 3 ค่า 3 รูปแบบ เพื่อดูระยะเวลาการดูค่าข้อมูลย้อนหลังหลายๆแบบและเปรียบเทียบ MSE
BH1, BH2, BH3 = backHistory[1], backHistory[3], backHistory[4]
return_period = 2
Y = base.shift(-return_period)
X4_BH1 = base.diff( BH1).shift( - BH1)
X4_BH2 = base.diff( BH2).shift( - BH2)
X4_BH3 = base.diff( BH3).shift( - BH3)
X4 = pd.concat([X4_BH1, X4_BH2, X4_BH3], axis=1)
X4.columns = ['MSFT_3DT', 'MSFT_6DT', 'MSFT_12DT']
X4 = pd.DataFrame(standard_scaler.fit_transform(X4.values), index = X4.index,columns=X4.columns)

In [21]:
# Forming Dataset
X = pd.concat([X1, X2, X3, X4], axis=1)
data = pd.concat([Y, X], axis=1)
data

,"(Adj Close, MSFT)","(Adj Close, GOOGL)","(Adj Close, IBM)",DEXJPUS,DEXUSUK,SP500,DJIA,VIXCLS,MSFT_3DT,MSFT_6DT,MSFT_12DT
2017-09-25,69.393135,-1.159659,-0.510365,-0.028635,0.589904,-1.286788,-1.525064,-1.185618,0.238255,0.144433,-0.149201
2017-09-26,69.411919,-1.154671,-0.462377,0.039003,0.530386,-1.286526,-1.527919,-1.190225,0.154603,-0.000577,-0.133526
2017-09-27,69.994499,-1.119086,-0.524971,0.117046,0.494676,-1.271704,-1.514242,-1.224782,0.167801,0.104649,-0.185001
2017-09-28,70.107262,-1.111310,-0.524971,0.104039,0.552493,-1.267315,-1.504422,-1.261642,0.171609,0.037902,-0.165468
2017-09-29,69.778389,-1.097200,-0.565305,0.101437,0.496376,-1.253801,-1.498628,-1.266250,-0.034634,-0.162232,-0.182281
...,...,...,...,...,...,...,...,...,...,...,...
2022-09-16,242.449997,0.616757,1.088542,4.055628,-2.875734,0.713760,0.542863,0.667768,NaN,NaN,NaN
2022-09-19,238.949997,0.625309,1.130989,NaN,NaN,0.752356,0.590705,0.605566,NaN,NaN,NaN
2022-09-20,240.076599,0.564180,0.999037,NaN,NaN,0.688475,0.514683,0.766830,NaN,NaN,NaN
2022-09-21,NaN,0.505268,0.872622,NaN,NaN,0.592565,0.387971,0.862437,NaN,NaN,NaN


In [22]:
# Drop NA
data.dropna(inplace=True)
# View Statistics
data.describe()

,"(Adj Close, MSFT)","(Adj Close, GOOGL)","(Adj Close, IBM)",DEXJPUS,DEXUSUK,SP500,DJIA,VIXCLS,MSFT_3DT,MSFT_6DT,MSFT_12DT
count,1061.000000,1061.000000,1061.000000,1061.000000,1061.000000,1061.000000,1061.000000,1061.000000,1061.000000,1061.000000,1061.000000
mean,165.971046,-0.205567,-0.223157,-0.338575,0.130736,-0.197839,-0.184988,-0.101468,0.126963,0.151380,0.001336
std,73.329604,0.921420,0.874140,0.357050,0.913578,0.931764,0.950400,1.034052,0.889761,0.875858,0.994074
min,69.393135,-1.159659,-3.287363,-1.214892,-2.751596,-1.663539,-2.423450,-1.308869,-3.831992,-3.763270,-3.677352
25%,102.842400,-0.862360,-0.698895,-0.593148,-0.461011,-0.901739,-0.865044,-0.779002,-0.269026,-0.231829,-0.412102
50%,140.505402,-0.647902,-0.249163,-0.326500,0.052543,-0.590221,-0.510803,-0.362019,0.091113,0.087783,0.053604
75%,214.686005,0.139127,0.261144,-0.081964,0.885793,0.416846,0.347160,0.232355,0.532482,0.590207,0.625610
max,340.882782,2.106611,1.897224,0.554088,2.077850,2.055378,1.992546,7.163257,2.577420,2.210529,2.421236


In [23]:
# Assign X, Y (drop datetime index)
Y = data[data.columns[0]]
X = data[data.columns[1:]]
print(Y)
print(X)
# feature selection (correlation)
# Calculate correlation between variables for only continuous data columns
corr_data = X.corr()
# Reduce Corr() to Lower Matrix
lower_tri = corr_data.where(np.tril(np.ones(corr_data.shape),k=-1).astype(np.bool))
lower_tri.fillna(0, inplace=True)
# Drop columns if |correlation value| > 0.9
to_drop = [column for column in lower_tri.columns if any(lower_tri[column] > 0.9)]

X.drop(columns=to_drop, inplace=True)
X

2017-09-25     69.393135
2017-09-26     69.411919
2017-09-27     69.994499
2017-09-28     70.107262
2017-09-29     69.778389
                 ...    
2021-12-28    337.117432
2021-12-29    334.136902
2021-12-30    332.577057
2022-01-03    314.326324
2022-01-04    311.842560
Name: (Adj Close, MSFT), Length: 1061, dtype: float64
            (Adj Close, GOOGL)  (Adj Close, IBM)   DEXJPUS   DEXUSUK  \
2017-09-25           -1.159659         -0.510365 -0.028635  0.589904   
2017-09-26           -1.154671         -0.462377  0.039003  0.530386   
2017-09-27           -1.119086         -0.524971  0.117046  0.494676   
2017-09-28           -1.111310         -0.524971  0.104039  0.552493   
2017-09-29           -1.097200         -0.565305  0.101437  0.496376   
...                        ...               ...       ...       ...   
2021-12-28            2.006793          1.143877  0.375889  0.547391   
2021-12-29            2.005780          1.207929  0.404505  0.620513   
2021-12-30            1

,"(Adj Close, IBM)",DEXJPUS,DEXUSUK,DJIA,VIXCLS,MSFT_3DT,MSFT_6DT,MSFT_12DT
2017-09-25,-0.510365,-0.028635,0.589904,-1.525064,-1.185618,0.238255,0.144433,-0.149201
2017-09-26,-0.462377,0.039003,0.530386,-1.527919,-1.190225,0.154603,-0.000577,-0.133526
2017-09-27,-0.524971,0.117046,0.494676,-1.514242,-1.224782,0.167801,0.104649,-0.185001
2017-09-28,-0.524971,0.104039,0.552493,-1.504422,-1.261642,0.171609,0.037902,-0.165468
2017-09-29,-0.565305,0.101437,0.496376,-1.498628,-1.266250,-0.034634,-0.162232,-0.182281
...,...,...,...,...,...,...,...,...
2021-12-28,1.143877,0.375889,0.547391,1.895183,-0.341285,-2.968232,-3.241830,-3.638811
2021-12-29,1.207929,0.404505,0.620513,1.917113,-0.409246,-3.355253,-3.680574,-3.677352
2021-12-30,1.257749,0.430520,0.663026,1.895151,-0.365474,-3.831992,-3.375453,-3.608294
2022-01-03,1.447236,0.443527,0.610310,1.940500,-0.449562,-3.021683,-3.763270,-3.638008


In [24]:
# Train / Test Preparation (try 2 Option)
# Option#1
test_size = (0.3 * len( X )).__round__()
train_size = (0.7 * len( X )).__round__()
X_train, X_test = X[0:train_size], X[train_size:len(X)]
Y_train, Y_test = Y[0:train_size], Y[train_size:len(X)]
# Option #2
seed = 4
# X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.3, random_state=seed)
X_train

,"(Adj Close, IBM)",DEXJPUS,DEXUSUK,DJIA,VIXCLS,MSFT_3DT,MSFT_6DT,MSFT_12DT
2017-09-25,-0.510365,-0.028635,0.589904,-1.525064,-1.185618,0.238255,0.144433,-0.149201
2017-09-26,-0.462377,0.039003,0.530386,-1.527919,-1.190225,0.154603,-0.000577,-0.133526
2017-09-27,-0.524971,0.117046,0.494676,-1.514242,-1.224782,0.167801,0.104649,-0.185001
2017-09-28,-0.524971,0.104039,0.552493,-1.504422,-1.261642,0.171609,0.037902,-0.165468
2017-09-29,-0.565305,0.101437,0.496376,-1.498628,-1.266250,-0.034634,-0.162232,-0.182281
...,...,...,...,...,...,...,...,...
2020-09-14,-0.910156,-0.803864,-0.398092,-0.143288,0.615933,0.255278,0.263458,0.302814
2020-09-15,-0.882220,-0.827277,-0.415097,-0.142738,0.585984,-0.091768,0.272215,0.161823
2020-09-16,-0.740142,-0.900118,-0.221239,-0.133817,0.637819,-0.040751,0.533179,0.412714
2020-09-17,-0.684271,-0.922230,-0.263751,-0.165444,0.686198,0.156146,0.641420,0.559670


GridSearchCV() 

In [28]:
# Create Model List
regression = { 'LR': LinearRegression(), 'SVR': SVR(), }

# Create Parameter Dictionary for Linear Regression
fit_intercept = [True, False]
normalize = [True, False]
params_LR = dict( fit_intercept = fit_intercept, normalize = normalize)

# Create Parameter Dictionary for SVR
kernel = ['linear', 'rbf', 'poly']
C_list = [10, 100]
ep_list = [0.1, 1, 5]
gamma = [0.01, 0.1]
degree = [2, 3]
params_SVR = dict( kernel = kernel, C = C_list, epsilon = ep_list, gamma = gamma, degree = degree )

# Cross Validation Model
# set k-fold crossvalidation with shuffle
num_fold = 3
seed = 500
kfold = KFold(n_splits=num_fold, shuffle = True, random_state=seed)
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# GridSearchCV() -> (a) 
for EST in regression:
    model = regression[EST]
    if (EST == 'LR'):
        params = params_LR
    else:
        params = params_SVR
    grid = GridSearchCV(estimator=model, 
                        n_jobs = 1, 
                        verbose = 10,
                        cv = kfold,
                        scoring = 'neg_mean_squared_error',
                        param_grid = params )
    grid_result = grid.fit(X_train, Y_train)

In [ ]:
# Show Best Parameters for both models
print('Best params: ',grid_result.best_params_)
print('Best score: ', grid_result.best_score_)

In [ ]:
# Show Score for each parameter combination for both model
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
 # Display Mean, std, params
grid_result.Bar()

RandomizedSearchCV()

In [ ]:
# Create Model List
regression = { 'LR': LinearRegression(), 'SVR': SVR(), }

In [ ]:
# Create Parameter Dictionary for Linear Regression
fit_intercept = [True, False]
normalize = [True, False]
params_LR = dict( fit_intercept = fit_intercept, normalize = normalize)

In [ ]:
# Create Parameter Dictionary for SVR
kernel = ['linear', 'rbf', 'poly']
C_list = list(np.linspace(0.1, 150, 5, dtype = float))
ep_list = list(np.linspace(0.1, 1, 5, dtype = float))
gamma = list(np.linspace(0.01, 0.1, 5, dtype = float))
degree = [2, 3]
params_SVR = dict( kernel = kernel, C = C_list, epsilon = ep_list, gamma = gamma, degree = degree )

# Cross Validation Model
# set k-fold crossvalidation with shuffle
num_fold = 3
seed = 500
kfold = KFold(n_splits=num_fold, shuffle = True, random_state=seed)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Show scatter plot compare y_test vs each model prediction
plt.scatter()

In [ ]:
# GridSearchCV() -> (a) 
for EST in regression:
    model = regression[EST]
    if (EST == 'LR'):
        params = params_LR
    else:
        params = params_SVR
    grid = GridSearchCV(estimator=model, 
                        n_jobs = 1, 
                        verbose = 10,
                        cv = kfold,
                        scoring = 'neg_mean_squared_error',
                        param_grid = params )
    grid_result = grid.fit(X_train, Y_train)

In [ ]:
# Show Best Parameters for both models
print('Best params: ',grid_result.best_params_)
print('Best score: ', grid_result.best_score_)

In [ ]:
# Show Score for each parameter combination for both model
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Display Mean, std, params
grid_result.Bar()

In [ ]:
# Show scatter plot compare y_test vs each model prediction
plt.scatter()